In [1]:
# импорт необходимых элементов
import statistics
from scipy.stats import iqr
import json
from datetime import datetime
with open('to_model/parametres.json', 'r') as f:
    param_json = json.load(f)

In [2]:
import dill
with open('models/model_pca_dill.pkl', 'rb') as f:
    pipeline = dill.load(f)
pipeline.steps

[('ful', Preprocess_ful()),
 ('transform',
  FeatureUnion(transformer_list=[('dummy_15',
                                  Pipeline(steps=[('column_dummy',
                                                   Preprocessing(aggfunc='median',
                                                                 column='15',
                                                                 norm='norm')),
                                                  ('one_hot',
                                                   OneHotEncoder(handle_unknown='ignore',
                                                                 sparse=False))])),
                                 ('dummy_16',
                                  Pipeline(steps=[('column_dummy',
                                                   Preprocessing(aggfunc='median',
                                                                 column='16',
                                                                 norm='norm')),
             

In [3]:
import pandas as pd
import numpy as np
from sklearn.metrics import f1_score

In [5]:
data = pd.read_csv('git_ignore_files/data_merge_asof.csv')
data

,Unnamed: 0_x,id,vas_id,buy_time,target,Unnamed: 0_y,0,1,2,3,...,243,244,245,246,247,248,249,250,251,252
0,578818,2582523,2.0,1531083600,0.0,3360134,314.560029,9.290888,342.989214,7.523202,...,6488.626154,-574.770792,-24.996269,121.369552,142.252276,-16.832889,-0.694428,-11.175933,-0.45614,0.0
1,603994,1292549,2.0,1531083600,0.0,2915346,93.880029,-217.499112,79.939214,-270.106798,...,-977.373846,-613.770792,-25.996269,-37.630448,-306.747724,-25.832889,-0.694428,-12.175933,-0.45614,0.0
2,293389,4053116,1.0,1531083600,0.0,1569112,125.110029,152.190888,111.169214,107.213202,...,-956.373846,-613.770792,-18.996269,761.369552,-213.747724,26.167111,-0.694428,39.824067,-0.45614,1.0
3,293387,4158361,2.0,1531083600,0.0,597622,-7.829971,-266.839112,-20.500786,-304.196798,...,1257.626154,495.229208,-19.996269,-37.630448,-108.747724,-25.832889,-0.694428,-12.175933,-0.45614,1.0
4,254350,3754468,4.0,1531083600,0.0,2376979,83.620029,535.610888,72.219214,503.343202,...,-977.373846,-613.770792,-25.996269,-37.630448,-306.747724,-25.832889,-0.694428,-12.175933,-0.45614,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
831648,535199,555080,5.0,1546203600,0.0,2043207,-96.799971,-408.179112,-110.740786,-460.786798,...,-977.373846,-613.770792,-25.996269,-37.630448,-306.747724,-25.832889,-0.694428,-12.175933,-0.45614,0.0
831649,535198,1729471,5.0,1546203600,0.0,286598,-86.209971,-397.589112,-100.150786,-450.196798,...,-977.373846,-613.770792,-25.996269,-37.630448,-306.747724,-25.832889,-0.694428,-12.175933,-0.45614,0.0
831650,535136,3676177,2.0,1546203600,0.0,2151141,-96.799971,-408.179112,-110.740786,-460.786798,...,-977.373846,-613.770792,-25.996269,-37.630448,-306.747724,-25.832889,-0.694428,-12.175933,-0.45614,0.0
831651,535090,2255038,2.0,1546203600,0.0,311341,-96.799971,49.450888,-104.390786,219.293202,...,-977.373846,-613.770792,-25.996269,-37.630448,-306.747724,-25.832889,-0.694428,-12.175933,-0.45614,0.0


In [6]:
from save_file_param import save_json

In [7]:
# сохраняем номера услуг для абонента

uslugi = data['vas_id'].unique()
uslugi
save_json('uslugi', [i for i in uslugi])

'Записано'

In [8]:
# вероятность подключения той или иной услуги для каждого абонента

df = pd.DataFrame()
for i in uslugi:
    data['vas_id'] = i
    df[i] = pipeline.predict_proba(data)[:, 1]
df.head(10)

,2.0,1.0,4.0,9.0,6.0,5.0,8.0,7.0
0,0.356613,0.355346,0.633881,0.354162,0.635235,0.361180,0.354162,0.354162
1,0.346127,0.348319,0.588292,0.344231,0.635235,0.348319,0.344231,0.344231
2,0.365173,0.364282,0.588760,0.358819,0.635235,0.364282,0.358819,0.358819
3,0.347447,0.354002,0.599090,0.345547,0.470282,0.355424,0.345547,0.345547
4,0.386347,0.386343,0.574412,0.386347,0.629585,0.386343,0.386347,0.386347
5,0.359776,0.362447,0.636155,0.355646,0.635235,0.362447,0.355646,0.355646
6,0.358397,0.355453,0.629110,0.357126,0.635235,0.360868,0.357126,0.357126
7,0.358397,0.360868,0.606104,0.357126,0.524520,0.360868,0.357126,0.357126
8,0.358930,0.387974,0.580165,0.356471,0.540305,0.360625,0.356471,0.356471
9,0.359776,0.359969,0.623136,0.359317,0.635235,0.359969,0.359317,0.359317


In [9]:
# вывод в датафрайме наиболее рекомендуемую услугу для предложения

recomendate = pd.DataFrame()
recomendate['id'] = data['id']
recomendate['рекомендуемая услуга'] = df.idxmax(axis=1)
recomendate

,id,рекомендуемая услуга
0,2582523,6.0
1,1292549,6.0
2,4053116,6.0
3,4158361,4.0
4,3754468,6.0
...,...,...
831648,555080,6.0
831649,1729471,6.0
831650,3676177,6.0
831651,2255038,6.0


In [10]:
# посчитаем сколько рекомендаций по классам 

for i in recomendate['рекомендуемая услуга'].unique():
    x = recomendate[recomendate['рекомендуемая услуга'] == i].shape[0]
    print(f'Рекомендуется услуга {i} для {x} абонентов')

Рекомендуется услуга 6.0 для 749050 абонентов
Рекомендуется услуга 4.0 для 79896 абонентов
Рекомендуется услуга 5.0 для 332 абонентов
Рекомендуется услуга 1.0 для 136 абонентов
Рекомендуется услуга 9.0 для 2113 абонентов
Рекомендуется услуга 7.0 для 12 абонентов
Рекомендуется услуга 8.0 для 57 абонентов
Рекомендуется услуга 2.0 для 57 абонентов


# рекомендации услуг где 1 наиболее рекомендуемая,чем выше цифра колонки-тем менее рекомендуемая

In [11]:
# датафрам с рекомендациями услуг по приоритету

from tqdm import tqdm
recomendate_sorted = pd.DataFrame(columns=[1,2,3,4,5,6,7,8])
for i in tqdm(df.index):
    a = pd.DataFrame()
    a['0'] = df.loc[i]
    recomendate_sorted.loc[recomendate_sorted.shape[0]] = a.sort_values('0', ascending=False).index
    
recomendate_sorted

100%|██████████| 831653/831653 [2:04:49<00:00, 111.05it/s] 


,1,2,3,4,5,6,7,8
0,6.0,4.0,5.0,2.0,1.0,9.0,8.0,7.0
1,6.0,4.0,1.0,5.0,2.0,9.0,8.0,7.0
2,6.0,4.0,2.0,1.0,5.0,9.0,8.0,7.0
3,4.0,6.0,5.0,1.0,2.0,9.0,8.0,7.0
4,6.0,4.0,2.0,9.0,8.0,7.0,1.0,5.0
...,...,...,...,...,...,...,...,...
831648,6.0,4.0,9.0,1.0,2.0,8.0,7.0,5.0
831649,6.0,4.0,9.0,8.0,2.0,7.0,1.0,5.0
831650,6.0,4.0,9.0,8.0,2.0,7.0,1.0,5.0
831651,6.0,4.0,9.0,2.0,8.0,7.0,1.0,5.0


In [12]:
# объединение с ID абонентов

for i in recomendate_sorted.columns:
    recomendate[i] = recomendate_sorted[i]
recomendate.drop('рекомендуемая услуга', axis=1, inplace=True)
recomendate

,id,рекомендуемая услуга,1,2,3,4,5,6,7,8
0,2582523,6.0,6.0,4.0,5.0,2.0,1.0,9.0,8.0,7.0
1,1292549,6.0,6.0,4.0,1.0,5.0,2.0,9.0,8.0,7.0
2,4053116,6.0,6.0,4.0,2.0,1.0,5.0,9.0,8.0,7.0
3,4158361,4.0,4.0,6.0,5.0,1.0,2.0,9.0,8.0,7.0
4,3754468,6.0,6.0,4.0,2.0,9.0,8.0,7.0,1.0,5.0
...,...,...,...,...,...,...,...,...,...,...
831648,555080,6.0,6.0,4.0,9.0,1.0,2.0,8.0,7.0,5.0
831649,1729471,6.0,6.0,4.0,9.0,8.0,2.0,7.0,1.0,5.0
831650,3676177,6.0,6.0,4.0,9.0,8.0,2.0,7.0,1.0,5.0
831651,2255038,6.0,6.0,4.0,9.0,2.0,8.0,7.0,1.0,5.0


In [ ]:
# что бы не прописывать постоянно - напишем функцию в отдельном файле

from predict_megafon import predict_megafon

out = predict_megafon(data)
out